# Chapter 2: Data Ingestion & Variables - Exercise

In [1]:
%reset
low_memory=False
import numpy as np
import pandas as pd

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## 2.1 Introduction & Problem Setting

Today you are diving into the wonderful world of **fraud detection**!

Our company was alerted to a new type of credit card fraud by neighbouring firms. The exact cause is unknown but the fraudster can make multiple transactions simultaneously (and therefor spend the same amount multiple times). As a promising new talent in the field of data processing we turn to you to find if our company is affected by this type of fraud and, if so, identify which of our clients is linked to the fraudulent transactions.

You receive 3 dataset from our company offices:

- London (al transactions UTC time): **Transactions_UTC_upload.txt**
- Chicago (US central time): **Transactions_US_Central_upload.txt**
- Shanghai: **Transactions_China_upload.txt**

The client data can be found in **Credit_customers.txt**

Can you give us the name of any credit card fraudsters using this new method, if any?

## 2.2 Data ingestion and exploration

First of all, load the different datasets into their own pandas dataframe. Analyse the headers, datatypes and data itself to get an overview.

In [2]:
Credit_clients = pd.read_csv("Credit_customers.txt", sep='\t', index_col = 0)
Credit_clients.head()

,Name
Client_ID,
1,"Braund, Mr. Owen Harris"
2,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
3,"Heikkinen, Miss. Laina"
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
5,"Allen, Mr. William Henry"


In [3]:
Credit_clients.dtypes

Name    object
dtype: object

In [4]:
US = pd.read_csv("Transactions_US_Central_upload.txt", sep = ";")
US.head()

,Client_ID,Transaction_time,time_zone
0,148,2019-12-15 05:24:35,US_Central
1,299,2019-12-15 13:09:57,US_Central
2,197,2019-12-15 13:30:02,US_Central
3,259,2019-12-15 14:26:03,US_Central
4,591,2019-12-16 06:31:55,US_Central


In [5]:
US.dtypes

Client_ID            int64
Transaction_time    object
time_zone           object
dtype: object

In [6]:
EU = pd.read_csv("Transactions_UTC_upload.txt", sep = ";")
EU.head()

,Client_ID,Transaction_time,time_zone
0,86,2019-12-15 13:39:21,UTC
1,313,2019-12-16 18:00:26,UTC
2,198,2019-12-17 06:20:07,UTC
3,466,2019-12-17 06:47:42,UTC
4,119,2019-12-17 11:46:27,UTC


In [7]:
EU.dtypes

Client_ID            int64
Transaction_time    object
time_zone           object
dtype: object

In [8]:
China = pd.read_csv("Transactions_China_upload.txt", sep = ";")
China.head()

,Client_ID,Transaction_time,time_zone
0,805,2019-12-15 03:37:36,China
1,67,2019-12-15 06:42:35,China
2,749,2019-12-15 14:48:02,China
3,828,2019-12-15 22:17:06,China
4,539,2019-12-15 23:30:35,China


In [9]:
China.dtypes

Client_ID            int64
Transaction_time    object
time_zone           object
dtype: object

## 2.3 Data Processing

Now, try and find the possible credit card fraudeurs!

*Hint: if you are stuck, don't be afraid to google some new ways to compare datasets or gather info ;)*

In [10]:
EU['Transaction_time'] = pd.to_datetime(EU['Transaction_time'], utc = True) 
US['Transaction_time'] = pd.to_datetime(US['Transaction_time']) 
China['Transaction_time'] = pd.to_datetime(China['Transaction_time']) 

In [11]:
China['Transaction_time'] = China['Transaction_time'].dt.tz_localize(tz = "Asia/Shanghai").dt.tz_convert('UTC')
US['Transaction_time'] = US['Transaction_time'].dt.tz_localize(tz = "US/central").dt.tz_convert('UTC')

In [27]:
transactions = pd.concat([EU, US, China], axis=0)

In [29]:
transactions.head()

,Client_ID,Transaction_time,time_zone
0,86,2019-12-15 13:39:21+00:00,UTC
1,313,2019-12-16 18:00:26+00:00,UTC
2,198,2019-12-17 06:20:07+00:00,UTC
3,466,2019-12-17 06:47:42+00:00,UTC
4,119,2019-12-17 11:46:27+00:00,UTC


In [32]:
duplicates = transactions['Transaction_time'].duplicated(keep=False)

In [33]:
transactions[duplicates]

,Client_ID,Transaction_time,time_zone
319,577,2020-03-02 04:59:43+00:00,UTC
307,577,2020-03-02 04:59:43+00:00,US_Central
336,577,2020-03-02 04:59:43+00:00,China


If you found any, link them to a known customer name.

In [34]:
Credit_clients.loc[577]

Name    Nigerian Prince
Name: 577, dtype: object